https://pythonrepo.com/repo/ShieldMnt-invisible-watermark

frequency methods

dwtDct: DWT + DCT transform, embed watermark bit into max non-trivial coefficient of block dct coefficents

dwtDctSvd: DWT + DCT transform, SVD decomposition of each block, embed watermark bit into singular value decomposition

rivaGan: encoder/decoder model with Attention mechanism + embed watermark bits into vector.

# Freq method
- dwtDct

In [1]:
#Install this for first time run
!pip install invisible-watermark
!pip install blind-watermark

In [2]:
import cv2
import numpy as np
import random
from skimage.util import random_noise
from blind_watermark import WaterMark
from imwatermark import WatermarkEncoder
from imwatermark import WatermarkDecoder

# blind_watermark

In [3]:
bwm1 = WaterMark(password_img=1, password_wm=1)
bwm1.read_img('1.png')
wm = 'test'
bwm1.read_wm(wm, mode='str')
bwm1.embed('11_blind.png')
len_wm = len(bwm1.wm_bit)
print('Put down the length of wm_bit {len_wm}'.format(len_wm=len_wm))

Put down the length of wm_bit 31


In [4]:
bwm1 = WaterMark(password_img=1, password_wm=1)
wm_extract = bwm1.extract('11_blind.png', wm_shape=len_wm, mode='str')
print(wm_extract)

test


# Freq method
- dwtDct

In [5]:
bgr = cv2.imread('1.png')
wm = 'test'

encoder = WatermarkEncoder()
encoder.set_watermark('bytes', wm.encode('utf-8'))
bgr_encoded = encoder.encode(bgr, 'dwtDct')

cv2.imwrite('11_wm.png', bgr_encoded)

True

In [ ]:
bgr = cv2.imread('11_wm.png')

decoder = WatermarkDecoder('bytes', 32)
watermark = decoder.decode(bgr, 'dwtDct')
print(watermark.decode('utf-8'))

test


# RivaGan Method

In [ ]:
bgr = cv2.imread('1.png')
wm = 'test'

encoder = WatermarkEncoder()
encoder.set_watermark('bytes', wm.encode('utf-8'))
encoder.loadModel()
bgr_encoded = encoder.encode(bgr, 'rivaGan')

cv2.imwrite('11_rg.png', bgr_encoded)

True

In [ ]:
bgr = cv2.imread('11_rg.png')

decoder = WatermarkDecoder('bytes', 32)
decoder.loadModel()
watermark = decoder.decode(bgr, 'rivaGan')
print(watermark.decode('utf-8'))

test


# DwtDctSvd

In [ ]:
bgr = cv2.imread('1.png')
wm = 'test'

encoder = WatermarkEncoder()
encoder.set_watermark('bytes', wm.encode('utf-8'))
bgr_encoded = encoder.encode(bgr, 'dwtDctSvd')

cv2.imwrite('11_svd.png', bgr_encoded)

True

In [ ]:
bgr = cv2.imread('11_svd.png')

decoder = WatermarkDecoder('bytes', 32)
watermark = decoder.decode(bgr, 'dwtDctSvd')
print(watermark.decode('utf-8'))

test


# Attack Model

In [ ]:
def gaussian(img, output_file_name):
    img_shape = img.shape
    gauss_img = random_noise(img, mode='gaussian', mean=0, var=0.01, clip= True)
    gauss_img = np.array(255*gauss_img, dtype = 'uint8')
    cv2.imwrite(output_file_name, gauss_img)

def salt_pepper(img, output_file_name):
    ratio = 0.1 #Probability of the noise
    snp_img = np.zeros(img.shape,np.uint8)
    thres = 1 - ratio
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            rdn = random.random()
            if rdn < ratio:
                snp_img[i][j] = 0
            elif rdn > thres:
                snp_img[i][j] = 255
            else:
                snp_img[i][j] = img[i][j]
    cv2.imwrite(output_file_name, snp_img)
    
def speckle(img, output_file_name):
    speckle_img = random_noise(img, mode='speckle', mean=0, var=0.01, clip= True)
    # The above function returns a floating-point image on the range [0, 1], thus we changed it to 'uint8' and from [0,255]
    speckle_img = np.array(255*speckle_img, dtype = 'uint8')
    cv2.imwrite(output_file_name, speckle_img)
    
def poisson(img, output_file_name):
    img_shape = img.shape
    noise = np.random.poisson(30, img_shape)
    output = img + noise
    cv2.imwrite(output_file_name, output)
    
def rotate(img, output_file_name, angle=10):
    rows, cols, _ = img.shape
    M = cv2.getRotationMatrix2D(center=(cols / 2, rows / 2), angle=angle, scale=1)
    output_img = cv2.warpAffine(img, M, (cols, rows))
    cv2.imwrite(output_file_name, output_img)
    
def upscale(img, output_file_name):
    img_shape = img.shape
    output_img = cv2.resize(img, None, fx=1.25, fy=1.25, interpolation = cv2.INTER_CUBIC)
    cv2.imwrite(output_file_name, output_img)
    
def downscale(img, output_file_name):
    img_shape = img.shape
    output_img = cv2.resize(img, None, fx=0.75, fy=0.75, interpolation = cv2.INTER_CUBIC)
    cv2.imwrite(output_file_name, output_img)
    
def averaging(img, output_file_name):
    img_shape = img.shape
    output_img = cv2.blur(img,(5,5)) 
    cv2.imwrite(output_file_name, output_img)
    
def gaussian_blurring(img, output_file_name):
    img_shape = img.shape
    output_img = cv2.GaussianBlur(img,(5,5),0) 
    cv2.imwrite(output_file_name, output_img)
    
def median_blurring(img, output_file_name):
    output_img = cv2.medianBlur(img,5)
    cv2.imwrite(output_file_name, output_img)
    
def bilateral_filtering(img, output_file_name):
    output_img = cv2.bilateralFilter(img,9,75,75)
    cv2.imwrite(output_file_name, output_img)
    
def sharpen_filtering(img, output_file_name):
    sharpen_filter = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharped_img = cv2.filter2D(img, -1, sharpen_filter)
    cv2.imwrite(output_file_name, sharped_img)

def crop_horizontal(img, output_file_name):
    ratio = 0.5  #Only left 20%
    img_shape = img.shape
    height = int(img_shape[0] * ratio)
    horizontal_img = img[:height, :, :]
    cv2.imwrite(output_file_name, horizontal_img)

def crop_vertical(img, output_file_name):
    ratio = 0.5
    img_shape = img.shape
    vertical = int(img_shape[1] * ratio)
    vertical_img = img[:, :vertical, :]
    cv2.imwrite(output_file_name, vertical_img)

def increase_brightness(img, output_file_name):
    ratio = 1.1 # +50% brightness
    inc_bright_img = img * ratio
    inc_bright_img[inc_bright_img > 255] = 255
    cv2.imwrite(output_file_name, inc_bright_img)
    
def decrease_brightness(img, output_file_name):
    ratio = 0.9 # -50% brightness
    dec_bright_img = img * ratio
    dec_bright_img[dec_bright_img > 255] = 255
    cv2.imwrite(output_file_name, dec_bright_img)
    
def masks(img, output_file_name):
    n = 5 # Set 2 mask
    ratio = 0.3 # mask ratio
    input_img_shape = img.shape
    mask_img = img.copy()
    for i in range(n):
        tmp = np.random.rand() * (1 - ratio)  # random one place to put mask，1-ratio to avoid overfloat
        start_height, end_height = int(tmp * input_img_shape[0]), int((tmp + ratio) * input_img_shape[0])
        tmp = np.random.rand() * (1 - ratio)
        start_width, end_width = int(tmp * input_img_shape[1]), int((tmp + ratio) * input_img_shape[1])

        mask_img[start_height:end_height, start_width:end_width, :] = 0
    cv2.imwrite(output_file_name, mask_img)

# Main 

In [ ]:
def attack_method(input_image, image_name):
    print('Attacking ' + image_name)

    algorithmn = []

    for i in algorithmn:

    gaussian(input_image, 'Attack/MRI_1/Algo_A/Guassian.png')
    salt_pepper(input_image, 'Attack/MRI_1/Algo_A/Salt_pepper.png')
    speckle(input_image, 'Attack/MRI_1/Algo_A/Speckle.png')
    poisson(input_image, 'Attack/MRI_1/Algo_A/Poisson.png')
    rotate(input_image, 'Attack/MRI_1/Algo_A/Rotate.png', angle=10)
    upscale(input_image, 'Attack/MRI_1/Algo_A/Upscale.png')
    downscale(input_image, 'Attack/MRI_1/Algo_A/Downscale.png')
    averaging(input_image, 'Attack/MRI_1/Algo_A/Averaging.png')
    gaussian_blurring(input_image, 'Attack/MRI_1/Algo_A/Gaussian_blurring.png')
    median_blurring(input_image, 'Attack/MRI_1/Algo_A/Median_blurring.png')
    bilateral_filtering(input_image, 'Attack/MRI_1/Algo_A/Bilateral_filtering.png')
    sharpen_filtering(input_image, 'Attack/MRI_1/Algo_A/Sharpen_filtering.png')
    crop_horizontal(input_image, 'Attack/MRI_1/Algo_A/Crop_horizontal.png')
    crop_vertical(input_image, 'Attack/MRI_1/Algo_A/Crop_vertical.png')
    increase_brightness(input_image, 'Attack/MRI_1/Algo_A/Increase_brightness.png')
    decrease_brightness(input_image, 'Attack/MRI_1/Algo_A/Decrease_brightness.png')
    masks(input_image, 'Attack/MRI_1/Algo_A/Masks.png')
    cv2.imwrite('Attack/MRI_1/Algo_A/' + image_name.split('.')[0] + '.jpg', input_image)


In [ ]:
wm_list = ['11_blind.png', '11_wm.png', '11_rg.png', '11_svd.png']

# imdir = 'Attack/MRI_1/Algo_A/'
# ext = ['png', 'jpg'] 

# files = []
# [files.extend(glob.glob(imdir + '*.' + e)) for e in ext]

# image_list = [(file) for file in files]

# algo=[Algo_A,Algo_B,AlgoC]
# for  x in algo:
# attack path = 'Attack/MRI_1'+x+''

def attacking(): 
    image_1_name = wm_list[0]
    image_2_name = wm_list[1]
    image_3_name = wm_list[2]
    image_4_name = wm_list[3]

    image_1 = cv2.imread(image_1_name)
    image_2 = cv2.imread(image_2_name)
    image_3 = cv2.imread(image_3_name)
    image_4 = cv2.imread(image_4_name)

    attack_method(image_1, image_1_name)
    attack_method(image_2, image_2_name)
    attack_method(image_3, image_3_name)
    attack_method(image_4, image_4_name)

    print('Done Attacking')

In [ ]:
def Watermarking(): 
    a = input("Image Watermarking \n 1. Encode the data \n 2. Attacking \n 3. Decode the data \n Your input is: ")
    userinput = int(a)
    if (userinput == 1):
        print("\nEncoding....")
#         encode_text() 
          
    elif (userinput == 2):
        print("\nAttacking....") 
        attacking()

    elif (userinput == 3):
        print("\nDecoding....") 
        # attacking()

    else: 
        raise Exception("Enter correct input") 

In [ ]:
Watermarking()

Image Watermarking 
 1. Encode the data 
 2. Attacking 
 3. Decode the data 
 Your input is: 2

Attacking....
Attacking 11_blind.png
Attacking 11_wm.png
Attacking 11_rg.png
Attacking 11_svd.png
Done Attacking
